### postprocess arise 

Alistair Duffey, October 2024

UKESM1

N.B.: runs over the CEDA archive data structure 

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import xarray as xr
from xmip.preprocessing import rename_cmip6
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

### options

# Model
model = 'UKESM1-0-LL'

# time-periods over which to take means
assessment_periods = {'SAI':slice('2050', '2069'),
                      'Background_warming':slice('2050', '2069'),
                      'Baseline':slice('2014', '2033'),
                      'Pre-industrial':slice('1850', '1900')}

# ARISE ensemble_members. We also only use these same members for the baseline
ens_mems = ['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2', 'r4i1p1f2', 'r8i1p1f2']


# all the CMIP7 BCVs which are monthly and on a single level:
# see markdown table below for details on variable meanings
vars_dict = {
             'prw':'Amon',
             'evspsbl':'Amon',
             'clivi':'Amon',
             'clt':'Amon',
             'clwvi':'Amon',
             'hfss':'Amon',
             'rlds':'Amon',
             'rsldscs':'Amon',
             'rlus':'Amon',
             'rlut':'Amon',
             'rlutcs':'Amon',
             'rsds':'Amon',
             'rsdscs':'Amon',
             'rsdt':'Amon',
             'rsus':'Amon',
             'rsuscs':'Amon',
             'rsut':'Amon',
             'rsutcs':'Amon',
             'pr':'Amon',
             'tas':'Amon',
             'uas':'Amon',
             'vas':'Amon',
             'hfls':'Amon',
             'hurs':'Amon',
             'huss':'Amon',
             'prc':'Amon',
             'prsn':'Amon',
             'ps':'Amon',
             'psl':'Amon',
             'sfcWind':'Amon',
             'tasmax':'Amon',
             'tasmin':'Amon',
             'tauu':'Amon',
             'tauv':'Amon',
             'ts':'Amon',
             'evspsblsoi':'Lmon',
             'lai':'Lmon',
             'mrfso':'Lmon',
             'mrro':'Lmon',
             'mrros':'Lmon',
             'mrso':'Lmon',
             'mrsos':'Lmon',
             'hfds':'Omon',
             'mlotst':'Omon',
             'sos':'Omon',
             'tauuo':'Omon',
             'tauvo':'Omon',
             'tos':'Omon',
             'zos':'Omon'
            }


# seasons
seasons = ['DJF', 'MAM', 'JJA', 'SON']

Variables used are as follows:

| Label          | Realm       | Title                                                    | Units       | Standard Name                                     |
|----------------|-------------|----------------------------------------------------------|-------------|---------------------------------------------------|
| Amon.prw       | Atmosphere  | Water Vapor Path                                         | kg m-2      | atmosphere_mass_content_of_water_vapor            |
| Amon.evspsbl   | Land        | Evaporation Including Sublimation and Transpiration      | kg m-2 s-1  | water_evapotranspiration_flux                     |
| Amon.clivi     | Atmosphere  | Ice Water Path                                           | kg m-2      | atmosphere_mass_content_of_cloud_ice              |
| Amon.clt       | Atmosphere  | Total Cloud Cover Percentage                             | %           | cloud_area_fraction                               |
| Amon.clwvi     | Atmosphere  | Condensed Water Path                                     | kg m-2      | atmosphere_mass_content_of_cloud_condensed_water  |
| Amon.hfss      | Surface     | Surface Upward Sensible Heat Flux                        | W m-2       | surface_upward_sensible_heat_flux                 |
| Amon.rlds      | Radiation   | Surface Downwelling Longwave Radiation                   | W m-2       | surface_downwelling_longwave_flux_in_air          |
| Amon.rldscs    | Radiation   | Surface Downwelling Clear-Sky Longwave Radiation         | W m-2       | surface_downwelling_longwave_flux_in_air_assuming_clear_sky |
| Amon.rlus      | Radiation   | Surface Upwelling Longwave Radiation                     | W m-2       | surface_upwelling_longwave_flux_in_air            |
| Amon.rlut      | Radiation   | TOA Outgoing Longwave Radiation                          | W m-2       | toa_outgoing_longwave_flux                        |
| Amon.rlutcs    | Radiation   | TOA Outgoing Clear-Sky Longwave Radiation                | W m-2       | toa_outgoing_longwave_flux_assuming_clear_sky     |
| Amon.rsds      | Radiation   | Surface Downwelling Shortwave Radiation                  | W m-2       | surface_downwelling_shortwave_flux_in_air         |
| Amon.rsdscs    | Radiation   | Surface Downwelling Clear-Sky Shortwave Radiation        | W m-2       | surface_downwelling_shortwave_flux_in_air_assuming_clear_sky |
| Amon.rsdt      | Radiation   | TOA Incident Shortwave Radiation                         | W m-2       | toa_incoming_shortwave_flux                       |
| Amon.rsus      | Radiation   | Surface Upwelling Shortwave Radiation                    | W m-2       | surface_upwelling_shortwave_flux_in_air           |
| Amon.rsuscs    | Radiation   | Surface Upwelling Clear-Sky Shortwave Radiation          | W m-2       | surface_upwelling_shortwave_flux_in_air_assuming_clear_sky |
| Amon.rsut      | Radiation   | TOA Outgoing Shortwave Radiation                         | W m-2       | toa_outgoing_shortwave_flux                       |
| Amon.rsutcs    | Radiation   | TOA Outgoing Clear-Sky Shortwave Radiation               | W m-2       | toa_outgoing_shortwave_flux_assuming_clear_sky    |
| Amon.pr        | Surface     | Precipitation                                            | kg m-2 s-1  | precipitation_flux                                |
| Amon.tas       | Surface     | Near-Surface Air Temperature                             | K           | air_temperature                                   |
| Amon.uas       | Surface     | Eastward Near-Surface Wind                               | m s-1       | eastward_wind                                     |
| Amon.vas       | Surface     | Northward Near-Surface Wind                              | m s-1       | northward_wind                                    |
| Amon.hfls      | Surface     | Surface Upward Latent Heat Flux                          | W m-2       | surface_upward_latent_heat_flux                   |
| Amon.hurs      | Surface     | Near-Surface Relative Humidity                           | %           | relative_humidity                                 |
| Amon.huss      | Surface     | Near-Surface Specific Humidity                           | 1           | specific_humidity                                 |
| Amon.prc       | Surface     | Convective Precipitation                                 | kg m-2 s-1  | convective_precipitation_flux                     |
| Amon.prsn      | Surface     | Snowfall Flux                                           | kg m-2 s-1  | snowfall_flux                                     |
| Amon.ps        | Surface     | Surface Air Pressure                                    | Pa          | surface_air_pressure                              |
| Amon.psl       | Surface     | Sea Level Pressure                                      | Pa          | air_pressure_at_mean_sea_level                    |
| Amon.sfcWind   | Surface     | Near-Surface Wind Speed                                 | m s-1       | wind_speed                                        |
| Amon.tasmax    | Surface     | Daily Maximum Near-Surface Air Temperature              | K           | air_temperature                                   |
| Amon.tasmin    | Surface     | Daily Minimum Near-Surface Air Temperature              | K           | air_temperature                                   |
| Amon.tauu      | Surface     | Surface Downward Eastward Wind Stress                   | Pa          | surface_downward_eastward_stress                  |
| Amon.tauv      | Surface     | Surface Downward Northward Wind Stress                  | Pa          | surface_downward_northward_stress                 |
| Amon.ts        | Surface     | Surface Temperature                                     | K           | surface_temperature                               |
| Lmon.evspsblsoi| Land        | Water Evaporation from Soil                             | kg m-2 s-1  | water_evaporation_flux_from_soil                  |
| Lmon.lai       | Land        | Leaf Area Index                                         | 1           | leaf_area_index                                   |
| Lmon.mrfso     | Land        | Soil Frozen Water Content                               | kg m-2      | soil_frozen_water_content                         |
| Lmon.mrro      | Land        | Total Runoff                                            | kg m-2 s-1  | runoff_flux                                       |
| Lmon.mrros     | Land        | Surface Runoff                                          | kg m-2 s-1  | surface_runoff_flux                               |
| Lmon.mrso      | Land        | Total Soil Moisture Content                             | kg m-2      | mass_content_of_water_in_soil                     |
| Lmon.mrsos     | Land        | Moisture in Upper Portion of Soil Column                | kg m-2      | mass_content_of_water_in_soil_layer               |
| Omon.hfds      | Ocean       | Downward Heat Flux at Sea Water Surface                 | W m-2       | surface_downward_heat_flux_in_sea_water           |
| Omon.mlotst    | Ocean       | Ocean Mixed Layer Thickness Defined by Sigma T          | m           | ocean_mixed_layer_thickness_defined_by_sigma_t    |
| Omon.sos       | Ocean       | Sea Surface Salinity                                    | 0.001       | sea_surface_salinity                              |
| Omon.tauuo     | Ocean       | Sea Water Surface Downward X Stress                     | N m-2       | downward_x_stress_at_sea_water_surface            |
| Omon.tauvo     | Ocean       | Sea Water Surface Downward Y Stress                     | N m-2       | downward_y_stress_at_sea_water_surface            |
| Omon.tos       | Ocean       | Sea Surface Temperature                                 | degC        | sea_surface_temperature                           |
| Omon.zos       | Ocean       | Sea Surface Height Above Geoid                          | m           | sea_surface_height_above_geoid                    |


In [2]:
### get historical-SSP2-4.5 and combine along the time dimension

def get_historical_ssp245_ds(variable, table='Amon'):
    """ gets historical and SSP2-4.5 and combine along the time dimension
        returns dataset with 5 members, each running over the historical 
        1850-2014 and ssp245 2015-2100 """
    
    ds_list = []
    for es in ens_mems:
        path = '/badc/cmip6/data/CMIP6/ScenarioMIP/MOHC/UKESM1-0-LL/ssp245/{e}/{t}/{v}/*/latest/'.format(e=es, t=table,v=variable)
        ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
        
        path_hist = glob.glob('/badc/cmip6/data/CMIP6/*/*/UKESM1-0-LL/historical/{e}/{t}/{v}/*/latest/'.format(
        t=table, v=variable, e=es))[0]
        ds_hist = rename_cmip6(xr.open_mfdataset(path_hist+'*.nc'))    
        ds = xr.concat([ds_hist, ds], dim='time')
        if 'height' in ds.variables:
            ds = ds.drop_vars('height')
        if 'type' in ds.variables:
            ds = ds.drop_vars('type')
        ds_list.append(ds)
    
    DS = xr.concat(ds_list, dim='Ensemble_member')
    return DS

## for ARISE
def get_ARISE_UKESM(variable='tas', table='Amon'):
    ds_list = []
    paths = glob.glob('/badc/deposited2022/arise/data/ARISE/MOHC/UKESM1-0-LL/arise-sai-1p5/*/{t}/{v}/*/*/'.format(
    t=table, v=variable))
    for path in paths:
        ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
        if 'height' in ds.variables:
            ds = ds.drop_vars('height')
        if 'type' in ds.variables:
            ds = ds.drop_vars('type')
        ds_list.append(ds)
    DS = xr.concat(ds_list, dim='Ensemble_member')
    return DS

def get_time_period(ds, slice_label):
    ds_out = ds.sel(time=assessment_periods[slice_label])
    ds_out.attrs['t_bnds'] = str(assessment_periods[slice_label].start+'_'+assessment_periods[slice_label].stop)
    return ds_out

In [5]:
def process_and_save_maps(ds, ds_seasonal, var, table, label, seasons=seasons):
    """ 
    Inputs
    ds: a time resolved, quarterly resampled, spatial dataset, with an ensemble_member dimension
    label: 'SSP245_baseline', 'ARISE', or 'SSP245_background'. Defines naming of outputs. 
    
    Function saves the mean and standard deviation across the whole time+ens_mems combined dimension
    """
    path = 'pp_archive/ARISE/UKESM1-0-LL/maps/{l}/{t}/{v}/'.format(l=label, t=table, v=var)
    os.makedirs(path+'/std/', exist_ok=True)
    os.makedirs(path+'/mean/', exist_ok=True)
    
    t_bnds = ds_seasonal.t_bnds
    for season in seasons:
        ds_season = ds_seasonal.where(ds_seasonal.time.dt.season == season, drop=True)
        std = ds_season.std(dim=['time', 'Ensemble_member'])
        mean = ds_season.mean(dim=['time', 'Ensemble_member'])
        
        std.to_netcdf(path + '/std/' +'{v}_{l}_{s}_{t}_std.nc'.format(v=var, l=label, s=season, t=t_bnds))
        mean.to_netcdf(path + '/mean/' + '{v}_{l}_{s}_{t}_mean.nc'.format(v=var, l=label, s=season, t=t_bnds))

    t_bnds = ds.t_bnds
    # repeat for the annual mean:
    std = ds.std(dim=['time', 'Ensemble_member'])
    mean = ds.mean(dim=['time', 'Ensemble_member'])
    
    std.to_netcdf(path + '/std/' + '{v}_{l}_annual_{t}_std.nc'.format(v=var, l=label, t=t_bnds))
    mean.to_netcdf(path + '/mean/' + '{v}_{l}_annual_{t}_mean.nc'.format(v=var, l=label, t=t_bnds))
    return

In [ ]:
# MAIN

for var in tqdm(vars_dict.keys()):

    print(var)
    # get data
    ds_ssp = get_historical_ssp245_ds(var, table=vars_dict[var])
    ds_arise = get_ARISE_UKESM(var, table=vars_dict[var])
    ds_ssp_seasonal = ds_ssp.resample(time="QS-DEC").mean()
    ds_arise_seasonal = ds_arise.resample(time="QS-DEC").mean()

    # process into time slice means:
    ssp_baseline, ssp_baseline_seasonal = get_time_period(ds_ssp, 'Baseline'), get_time_period(ds_ssp_seasonal, 'Baseline')
    ssp_background, ssp_background_seasonal = get_time_period(ds_ssp, 'Background_warming'), get_time_period(ds_ssp_seasonal, 'Background_warming')
    arise_assmt, arise_assmt_seasonal = get_time_period(ds_arise, 'SAI'), get_time_period(ds_arise_seasonal, 'SAI')

    process_and_save_maps(ssp_baseline, ssp_baseline_seasonal, 
                 var=var, table=vars_dict[var], 
                 label='SSP245_baseline', seasons=seasons)
    process_and_save_maps(ssp_background, ssp_background_seasonal, 
                     var=var, table=vars_dict[var], 
                     label='SSP245_background', seasons=seasons)
    process_and_save_maps(arise_assmt, arise_assmt_seasonal, 
                     var=var, table=vars_dict[var], 
                     label='ARISE_assmt', seasons=seasons)


  0%|          | 0/49 [00:00<?, ?it/s]

prw


  2%|▏         | 1/49 [01:20<1:04:33, 80.70s/it]

evspsbl


  4%|▍         | 2/49 [02:54<1:09:12, 88.36s/it]

...

clwvi


 10%|█         | 5/49 [07:27<1:06:18, 90.43s/it]

hfss


 12%|█▏        | 6/49 [09:00<1:05:25, 91.29s/it]

rlds
